### Vector Stores - FAISS

1. FAISS
2. CHROMADB
3. ASTRADB (Cassandra DB)

FAISS - Facebook AI Similarity Search
- Search and clustering of dense vectors.
- Search in sets of vectors of any size, upto ones that do not fit in RAM.
- Contains supporting code and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [2]:
loader = TextLoader("speech.txt")

In [3]:
documents = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)

In [7]:
docs = text_splitter.split_documents(documents)

In [8]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [13]:
embeddings = OllamaEmbeddings(model='gemma:2b')

In [14]:
db = FAISS.from_documents(docs, embeddings)
db

In [15]:
# Querying
query = "What does the speaker believe is the main reason behind united states should enter the war?"
query1 = "How does the speaker describe the desired outcome of the war?"

docs = db.similarity_search(query1)
docs

[Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b4488dd0-df2e-475b-ad1a-136bd8def3df', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising

In [16]:
docs[0]

Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.')

In [17]:
docs[0].page_content

'â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

##### Retrival
We can also convert vectorstore into Retriever class. So, can easily use it in other Langchain methods, which largely work with retrievers. 

In [19]:
# Convert vectorstore into retriever and use it
retriever = db.as_retriever()
response = retriever.invoke(query1)

In [20]:
response

[Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b4488dd0-df2e-475b-ad1a-136bd8def3df', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising

##### Similarity Search with score
Some FAISS specific methods. One is similarity_search_with_score - returns document with distance score of the query to them.
Returned distance score is L2 distance. Lower score is better.

In [21]:
docs_and_sscore = db.similarity_search_with_score(query1)
docs_and_sscore

[(Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2954.4536)),
 (Document(id='b4488dd0-df2e-475b-ad1a-136bd8def3df', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of 

In [22]:
embeddings_vector = embeddings.embed_query(query1)
embeddings_vector

[0.29347342252731323,
 -2.0991556644439697,
 0.2276652604341507,
 0.9760229587554932,
 0.638526439666748,
 0.5283483862876892,
 -0.9287277460098267,
 0.10302824527025223,
 -0.08266652375459671,
 -0.7845128178596497,
 1.0946028232574463,
 -0.0028771650977432728,
 -1.079031229019165,
 1.0760583877563477,
 0.0697866827249527,
 -1.023376703262329,
 3.121046304702759,
 1.6792463064193726,
 1.1673504114151,
 0.7538043856620789,
 0.3069216012954712,
 -0.2727566063404083,
 0.3220255374908447,
 1.378890037536621,
 0.1787915974855423,
 -0.4122703969478607,
 -1.4097799062728882,
 -1.2852100133895874,
 -0.5455846190452576,
 -2.078129768371582,
 -0.26022592186927795,
 -1.4310917854309082,
 1.224184274673462,
 -0.9681358933448792,
 -0.39843571186065674,
 -0.26654911041259766,
 1.8904190063476562,
 0.5086583495140076,
 0.12508752942085266,
 -0.5380474328994751,
 0.3671032786369324,
 0.49337345361709595,
 0.9563034176826477,
 -1.3147554397583008,
 -1.3466671705245972,
 0.6260555386543274,
 0.061975494

In [23]:
docs_and_score = db.similarity_search_by_vector(embeddings_vector)
docs_and_score

[Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b4488dd0-df2e-475b-ad1a-136bd8def3df', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising

In [24]:
# Store vectorstore db in local
# Save and load

db.save_local('faiss_index')

In [26]:
# Load model

new_df = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization=True)

In [27]:
docs = new_df.similarity_search(query1)

In [28]:
docs

[Document(id='ca2b8195-d8fa-412c-899c-0b198c6f15c5', metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between usâ€”however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='b4488dd0-df2e-475b-ad1a-136bd8def3df', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising